# Visualize, Evaluate and Compare Performance

In [1]:
# table of contents needs to be updateddddd !!!!

## Table of Contents
- [1. Imports](#1-imports)
   - [1.1. Import Necessary Libraries](#11-import-necessary-libraries)
   - [1.2. Import Data](#12-import-data)
- [2. Visualizations](#2-visualizations)
   - [2.1. Average Best Fitness (per generation)](#21-average-best-fitness-(per-generation))
   - [2.2. Median Best Fitness (per generation)](#22-median-best-fitness-(per-generation))
   - [2.3. Best Fitness (per run)](#23-best-final-fitness-(per-run))
   - [2.4. Average Best Fitness of each mutation (per generation)](#24-average-best-fitness-of-each-mutation-(per-generation))
- [3. Metrics](#3-metrics)

## 1. Imports

### 1.1. Import Necessary Libraries

__`Step 1`__ Import necessary libraries.

In [2]:
import pandas as pd
from library.performance_analysis import *

### 1.2. Import Data

__`Step 2`__ Import the data from our runs.

In [3]:
df_exp1 = pd.read_csv('combination_search/final_results_exp1.csv')
df_exp2 = pd.read_csv('combination_search/final_results_exp2.csv')
df_exp3 = pd.read_csv('combination_search/final_results_exp3.csv')

#df_elitism = pd.read_csv('combination_search/elitism_exp.csv')

#print(df_exp1.head())
#print(df_exp2.head())
#print(df_exp3.head())

#CORRIGIR NOMES

## 2. Visualizations

### 2.1. Intra-Run Analysis: Median Best Fitness

__`Step 3`__ Plot the Median best fitness of each generation of each combination.

In [4]:
plot_ABF_MBF(df_exp1, exp_comb='1', metric='MBF')
plot_ABF_MBF(df_exp2, exp_comb='2', metric='MBF')
plot_ABF_MBF(df_exp3, exp_comb='3', metric='MBF')

### 2.2. Inter-Run Analysis: Stability of the combination

__`Step 4`__ Check ABF, STD and ABF/STD on all combinations to access stability.

In [6]:
get_abf_stats(df_exp1)[0]


Most Stable Comb: ('two_phase_shuffle_mutation', 'cyclic_crossover', 'tournament_selection')
Least Stable Comb: ('semi_shuffle', 'partially_mapped_crossover', 'linear_ranking_selection')


,Combination,ABF,STD,ABF/STD,ABF^2/STD - ABF
0,"('block_rotation_mutation', 'cyclic_crossover', 'linear_ranking_selection')",1.278492,0.041743,30.627655,37.878720
1,"('block_rotation_mutation', 'cyclic_crossover', 'tournament_selection')",1.436627,0.030591,46.962975,66.031651
2,"('block_rotation_mutation', 'partially_mapped_crossover', 'linear_ranking_selection')",1.258933,0.044113,28.538627,34.669277
3,"('block_rotation_mutation', 'partially_mapped_crossover', 'tournament_selection')",1.609877,0.044517,36.163397,56.608744
4,"('semi_shuffle', 'cyclic_crossover', 'linear_ranking_selection')",1.242853,0.043635,28.483168,34.157529
5,"('semi_shuffle', 'cyclic_crossover', 'tournament_selection')",1.427883,0.025023,57.063875,80.052635
6,"('semi_shuffle', 'partially_mapped_crossover', 'linear_ranking_selection')",1.256759,0.049787,25.242767,30.467324
7,"('semi_shuffle', 'partially_mapped_crossover', 'tournament_selection')",1.563082,0.049363,31.664938,47.931822
8,"('two_phase_shuffle_mutation', 'cyclic_crossover', 'linear_ranking_selection')",1.284225,0.029379,43.712327,54.852223
9,"('two_phase_shuffle_mutation', 'cyclic_crossover', 'tournament_selection')",1.407412,0.023752,59.255029,81.988807


In [ ]:
get_abf_stats(df_exp2)[0]

Most Stable Comb: ('block_rotation_mutation', 'cyclic_crossover', 'linear_ranking_selection')
Least Stable Comb: ('semi_shuffle', 'cyclic_crossover', 'linear_ranking_selection')


,Combination,ABF,STD,ABF/STD
0,"('block_rotation_mutation', 'cyclic_crossover', 'linear_ranking_selection')",1.272783,0.021282,59.805109
1,"('block_rotation_mutation', 'cyclic_crossover', 'tournament_selection')",1.394262,0.028393,49.105396
2,"('block_rotation_mutation', 'partially_mapped_crossover', 'linear_ranking_selection')",1.272120,0.033075,38.461154
3,"('block_rotation_mutation', 'partially_mapped_crossover', 'tournament_selection')",1.537450,0.033670,45.662073
4,"('semi_shuffle', 'cyclic_crossover', 'linear_ranking_selection')",1.263792,0.048971,25.807187
5,"('semi_shuffle', 'cyclic_crossover', 'tournament_selection')",1.467102,0.029891,49.081456
6,"('semi_shuffle', 'partially_mapped_crossover', 'linear_ranking_selection')",1.243384,0.039730,31.295865
7,"('semi_shuffle', 'partially_mapped_crossover', 'tournament_selection')",1.620897,0.046539,34.828884
8,"('two_phase_shuffle_mutation', 'cyclic_crossover', 'linear_ranking_selection')",1.303605,0.024813,52.538194
9,"('two_phase_shuffle_mutation', 'cyclic_crossover', 'tournament_selection')",1.362315,0.028431,47.916294


In [ ]:
get_abf_stats(df_exp3)[0]


Most Stable Comb: ('block_rotation_mutation', 'cyclic_crossover', 'tournament_selection')
Least Stable Comb: ('semi_shuffle', 'partially_mapped_crossover', 'linear_ranking_selection')


,Combination,ABF,STD,ABF/STD
0,"('block_rotation_mutation', 'cyclic_crossover', 'linear_ranking_selection')",1.252681,0.033109,37.835319
1,"('block_rotation_mutation', 'cyclic_crossover', 'tournament_selection')",1.470405,0.024453,60.132431
2,"('block_rotation_mutation', 'partially_mapped_crossover', 'linear_ranking_selection')",1.268954,0.038451,33.001957
3,"('block_rotation_mutation', 'partially_mapped_crossover', 'tournament_selection')",1.618567,0.047021,34.422370
4,"('semi_shuffle', 'cyclic_crossover', 'linear_ranking_selection')",1.246513,0.037457,33.278479
5,"('semi_shuffle', 'cyclic_crossover', 'tournament_selection')",1.476043,0.029899,49.367011
6,"('semi_shuffle', 'partially_mapped_crossover', 'linear_ranking_selection')",1.243417,0.047452,26.203828
7,"('semi_shuffle', 'partially_mapped_crossover', 'tournament_selection')",1.663101,0.056566,29.401202
8,"('two_phase_shuffle_mutation', 'cyclic_crossover', 'linear_ranking_selection')",1.307608,0.045268,28.885799
9,"('two_phase_shuffle_mutation', 'cyclic_crossover', 'tournament_selection')",1.380277,0.035188,39.225393


__`Step 5`__ Plot the best fitness of each run (for some combinations). Choosing to visualize the ones considered the best and least stable combination of each experiment.

In [ ]:
plot_BF_inter_run_boxplot(df_exp1, exp_comb='1')
plot_BF_inter_run_boxplot(df_exp2, exp_comb='2')
plot_BF_inter_run_boxplot(df_exp3, exp_comb='3')

In [ ]:
#only plot those of interest
plot_BF_inter_run(df_exp1, ['TCT', 'SPL'], '1')
plot_BF_inter_run(df_exp2, ['BPL', 'SPL'], '2')
plot_BF_inter_run(df_exp3, ['BCT', 'SPL'], '3')

### 2.3. Between Experiments: Comparison of probabilities

__`Step 6`__ Scatter the final results of each combination per experiment, and for each experiment by combination.


In [ ]:
exp_bf_per_comb({'exp1': 'Probability of Mutation: 0.1', 'exp2': 'Probability of Mutation: 0.2', 'exp3': 'Probability of Mutation: 0.3'})

In [ ]:
comb_bf_per_exp({'exp1': 'Probability of Mutation: 0.1', 'exp2': 'Probability of Mutation: 0.2', 'exp3': 'Mut: 0.3'})

__`Step 7`__ For each combination, plot the different experiments (with different probabilities), along with standart deviation.

In [ ]:
# Checking only the combinations that had the best fitness and most stability for each experiment

comb_between_exp(df_exps = {'exp1': 'Probability of Mutation: 0.1', 'exp2': 'Probability of Mutation: 0.2',
                            'exp3': 'Mut: 0.3'
                            }, comb_ids=['BPT', 'BPL'],
                            metric='MBF',
                            std=True
                            )#,'TPT', 'SPT', 'BPL', 'TPL', 'SPL'])

### 2.4. Plotting all together (per Combination+Experiment)

__`Step 8`__ By selecting x best combinations of each experiment, plot them all together, so we can have a full picture of combinations and their experiment (where it was only changed probabilities).

In [1]:
import pandas as pd
from library.performance_analysis import *

In [5]:
plot_Xbest_per_exp(Xbest=2, df_exps={'exp1': 'Prob Mut 0.1', 'exp2': 'Prob Mut 0.2', 'exp3': 'Customized Prob Muts'}, std=True)

In [191]:
plot_Xbest_per_exp_box(Xbest=3, metric='ABF',
                       df_exps={'exp1': 'Prob Mut 0.1', 'exp2': 'Prob Mut 0.2', 'exp3': 'Customized Prob Muts'})

### 2.5. Elitism Experiment: Comparison of Final Combination with and without elitism

__`Step 10`__ For the final combination, plot the best fitness per generation with and without the result.

In [ ]:
#elistim_tuning_plot(df_elitism)

## 3. Statistical tests

In [19]:
from scipy.stats import mannwhitneyu
from typing import List, Tuple, Dict, Any
def pairwise_mannwhitney(combs: List[Tuple], alpha:float = 0.05) -> Dict[str, Any]:
    """
    Perform pairwise Mann-Whitney U tests on the provided combinations.

    Parameters:
    - combs: List of tuples containing combinations to compare.
    - alpha: Significance level for the tests.

    Returns:
    - Dictionary with combinations as keys and p-values as values.
    """

    df_exp1 = pd.read_csv('combination_search/final_results_exp1.csv')
    df_exp2 = pd.read_csv('combination_search/final_results_exp2.csv')
    df_exp3 = pd.read_csv('combination_search/final_results_exp3.csv')

    data_combs = []
    for comb, exp in combs:
        print(f'Combination: {comb} - {exp}')
        if exp == 'exp1':
            df = df_exp1
        elif exp == 'exp2':
            df = df_exp2
        else:
            df = df_exp3

        # Get the last generation fitness values for all runs of this combination
        data = df[(df['Combination ID'] == comb) & (df['Generation'] == df['Generation'].max())]['Fitness'].values
        data_combs.append((comb, exp, data))
        print(f'Sample size: {len(data)}, Mean fitness: {data.mean():.4f}\n')

    # Perform pairwise Mann-Whitney U tests
    print("Pairwise Mann-Whitney U Tests:")
    print("-" * 40)

    results = {}
    for i in range(len(data_combs)):
        for j in range(i+1, len(data_combs)):
            comb1, exp1, data1 = data_combs[i]
            comb2, exp2, data2 = data_combs[j]
            
            stat, p_value = mannwhitneyu(data1, data2, alternative='two-sided')

            key = f"{comb1}({exp1}) vs {comb2}({exp2})"
            results[key] = {
                'statistic': stat,
                'p_value': p_value,
                'significant': p_value < alpha
            }
            
            print(f"Comparing {key}:")
            print(f'  Mann-Whitney U statistic: {stat:.2f}, p-value: {p_value:.4f}')
            
            if p_value < alpha:
                print(f"  Result: Significant difference (p < {alpha})")
            else:
                print(f"  Result: No significant difference (p > {alpha})")
            print()
            
    return results


In [17]:
from scipy.stats import mannwhitneyu, ranksums
from typing import List, Tuple, Dict, Any

def pairwise_sumrank_test(combs: List[Tuple], alpha:float = 0.05) -> Dict[str, Any]:
    """
    Perform pairwise Wilcoxon sum-rank tests on the provided combinations.

    Parameters:
    - combs: List of tuples containing combinations to compare (combination_id, experiment).
    - alpha: Significance level for the tests.

    Returns:
    - Dictionary with results of the tests.
    """

    df_exp1 = pd.read_csv('combination_search/final_results_exp1.csv')
    df_exp2 = pd.read_csv('combination_search/final_results_exp2.csv')
    df_exp3 = pd.read_csv('combination_search/final_results_exp3.csv')
    # We don't need to reimport these as they're already loaded in the notebook
    data_combs = []
    
    # Process each combination and extract data
    for comb, exp in combs:
        print(f'Combination: {comb} - {exp}')
        if exp == 'exp1':
            df = df_exp1
        elif exp == 'exp2':
            df = df_exp2
        else:
            df = df_exp3

        # Get the last generation fitness values for all runs of this combination
        data = df[(df['Combination ID'] == comb) & (df['Generation'] == df['Generation'].max())]['Fitness'].values
        data_combs.append((comb, exp, data))
        print(f'Sample size: {len(data)}, Mean fitness: {data.mean():.4f}\n')

    # Perform pairwise Wilcoxon sum-rank tests
    print("Pairwise Wilcoxon Sum-Rank Tests:")
    print("-" * 40)
    
    results = {}
    for i in range(len(data_combs)):
        for j in range(i+1, len(data_combs)):
            comb1, exp1, data1 = data_combs[i]
            comb2, exp2, data2 = data_combs[j]
            
            # Using ranksums instead of mannwhitneyu for Wilcoxon sum-rank test
            stat, p_value = ranksums(data1, data2)
            
            key = f"{comb1}({exp1}) vs {comb2}({exp2})"
            results[key] = {
                'statistic': stat,
                'p_value': p_value,
                'significant': p_value < alpha
            }
            
            print(f"Comparing {key}:")
            print(f'  Sum-Rank Z statistic: {stat:.4f}, p-value: {p_value:.4f}')
            
            if p_value < alpha:
                print(f"  Result: Significant difference (p < {alpha})")
            else:
                print(f"  Result: No significant difference (p > {alpha})")
            print()
    
    return results

### Effect of different probabilities in mutations

- #### Semi-shuffle:

In [ ]:
S_combs = [('SPT', 'exp1'), ('SPT', 'exp2'), ('SPT', 'exp3')]

In [ ]:
def non_parametric_pairwise_test

### Best combinations

In [8]:
# TOP 3 
best_comb = [('SPT', 'exp3'), ('SPT', 'exp2'), ('BPT', 'exp3'), ('BPT', 'exp1')]

In [12]:
pairwise_mannwhitney(best_comb, alpha=0.05)

Combination: SPT - exp3
Sample size: 30, Mean fitness: 1.6631

Combination: SPT - exp2
Sample size: 30, Mean fitness: 1.6209

Combination: BPT - exp3
Sample size: 30, Mean fitness: 1.6186

Combination: BPT - exp1
Sample size: 30, Mean fitness: 1.6099

Pairwise Mann-Whitney U Tests:
----------------------------------------
Comparing SPT(exp3) vs SPT(exp2):
  Mann-Whitney U statistic: 636.00, p-value: 0.0061
  Result: Significant difference (p < 0.05)

Comparing SPT(exp3) vs BPT(exp3):
  Mann-Whitney U statistic: 643.00, p-value: 0.0044
  Result: Significant difference (p < 0.05)

Comparing SPT(exp3) vs BPT(exp1):
  Mann-Whitney U statistic: 697.50, p-value: 0.0003
  Result: Significant difference (p < 0.05)

Comparing SPT(exp2) vs BPT(exp3):
  Mann-Whitney U statistic: 464.00, p-value: 0.8418
  Result: No significant difference (p > 0.05)

Comparing SPT(exp2) vs BPT(exp1):
  Mann-Whitney U statistic: 525.50, p-value: 0.2675
  Result: No significant difference (p > 0.05)

Comparing BPT(e

In [18]:
pairwise_sumrank_test(best_comb, alpha=0.05)

Combination: SPT - exp3
Sample size: 30, Mean fitness: 1.6631

Combination: SPT - exp2
Sample size: 30, Mean fitness: 1.6209

Combination: BPT - exp3
Sample size: 30, Mean fitness: 1.6186

Combination: BPT - exp1
Sample size: 30, Mean fitness: 1.6099

Pairwise Wilcoxon Sum-Rank Tests:
----------------------------------------
Comparing SPT(exp3) vs SPT(exp2):
  Sum-Rank Z statistic: 2.7499, p-value: 0.0060
  Result: Significant difference (p < 0.05)

Comparing SPT(exp3) vs BPT(exp3):
  Sum-Rank Z statistic: 2.8534, p-value: 0.0043
  Result: Significant difference (p < 0.05)

Comparing SPT(exp3) vs BPT(exp1):
  Sum-Rank Z statistic: 3.6591, p-value: 0.0003
  Result: Significant difference (p < 0.05)

Comparing SPT(exp2) vs BPT(exp3):
  Sum-Rank Z statistic: 0.2070, p-value: 0.8360
  Result: No significant difference (p > 0.05)

Comparing SPT(exp2) vs BPT(exp1):
  Sum-Rank Z statistic: 1.1162, p-value: 0.2643
  Result: No significant difference (p > 0.05)

Comparing BPT(exp3) vs BPT(exp1)

{'SPT(exp3) vs SPT(exp2)': {'statistic': np.float64(2.749903127951011),
  'p_value': np.float64(0.005961288528864798),
  'significant': np.True_},
 'SPT(exp3) vs BPT(exp3)': {'statistic': np.float64(2.853394105884651),
  'p_value': np.float64(0.004325494122990468),
  'significant': np.True_},
 'SPT(exp3) vs BPT(exp1)': {'statistic': np.float64(3.6591452912251357),
  'p_value': np.float64(0.00025305784556082746),
  'significant': np.True_},
 'SPT(exp2) vs BPT(exp3)': {'statistic': np.float64(0.2069819558672804),
  'p_value': np.float64(0.8360239504893114),
  'significant': np.False_},
 'SPT(exp2) vs BPT(exp1)': {'statistic': np.float64(1.116224119141405),
  'p_value': np.float64(0.26432621279678536),
  'significant': np.False_},
 'BPT(exp3) vs BPT(exp1)': {'statistic': np.float64(0.8648888870168503),
  'p_value': np.float64(0.38709977767912396),
  'significant': np.False_}}

In [18]:
from scipy.stats import ranksums
# Test ranksums between top 3 combinations

# Extract data for each of the top combinations
data_combs = []
for comb, exp in best_comb:
    print(f'Combination: {comb} - {exp}')
    if exp == 'exp1':
        df = df_exp1
    elif exp == 'exp2':
        df = df_exp2
    else:
        df = df_exp3

    # Get the last generation fitness values for all runs of this combination
    data = df[(df['Combination ID'] == comb) & (df['Generation'] == df['Generation'].max())]['Fitness'].values
    data_combs.append((comb, exp, data))
    print(f'Sample size: {len(data)}, Mean fitness: {data.mean():.4f}\n')

# Perform pairwise Wilcoxon rank-sum tests
print("Pairwise Wilcoxon Rank-Sum Tests:")
print("-" * 40)
for i in range(len(data_combs)):
    for j in range(i+1, len(data_combs)):
        comb1, exp1, data1 = data_combs[i]
        comb2, exp2, data2 = data_combs[j]
        
        stat, p_value = ranksums(data1, data2)
        
        print(f"Comparing {comb1}({exp1}) vs {comb2}({exp2}):")
        print(f'  Rank-Sum Z statistic: {stat:.4f}, p-value: {p_value:.4f}')
        
        alpha = 0.05
        if p_value < alpha:
            print(f"  Result: Significant difference (p < {alpha})")
        else:
            print(f"  Result: No significant difference (p > {alpha})")
        print()


Combination: SPT - exp3
Sample size: 30, Mean fitness: 1.6631

Combination: SPT - exp2
Sample size: 30, Mean fitness: 1.6209

Combination: BPT - exp3
Sample size: 30, Mean fitness: 1.6186

Combination: BPT - exp1
Sample size: 30, Mean fitness: 1.6099

Pairwise Wilcoxon Rank-Sum Tests:
----------------------------------------
Comparing SPT(exp3) vs SPT(exp2):
  Rank-Sum Z statistic: 2.7499, p-value: 0.0060
  Result: Significant difference (p < 0.05)

Comparing SPT(exp3) vs BPT(exp3):
  Rank-Sum Z statistic: 2.8534, p-value: 0.0043
  Result: Significant difference (p < 0.05)

Comparing SPT(exp3) vs BPT(exp1):
  Rank-Sum Z statistic: 3.6591, p-value: 0.0003
  Result: Significant difference (p < 0.05)

Comparing SPT(exp2) vs BPT(exp3):
  Rank-Sum Z statistic: 0.2070, p-value: 0.8360
  Result: No significant difference (p > 0.05)

Comparing SPT(exp2) vs BPT(exp1):
  Rank-Sum Z statistic: 1.1162, p-value: 0.2643
  Result: No significant difference (p > 0.05)

Comparing BPT(exp3) vs BPT(exp1)

In [ ]:


# Sample data - two independent groups
group1 = [20, 23, 21, 25, 18, 17, 18]
group2 = [24, 25, 21, 22, 23, 18, 28]

# Perform Wilcoxon rank-sum test
statistic, p_value = mannwhitneyu(group1, group2, 
                                  #alternative='less', use_continuity=False
                                  )

print(f"Test statistic: {statistic}")
print(f"P-value: {p_value}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis - distributions are different")
else:
    print("Fail to reject null hypothesis - no significant difference")

Test statistic: 12.5
P-value: 0.13864617169781115
Fail to reject null hypothesis - no significant difference


In [13]:
get_abf_stats(df_exp1)[1]#['ABF/STD']

Most Stable Comb: ('two_phase_shuffle_mutation', 'cyclic_crossover', 'tournament_selection')
Least Stable Comb: ('semi_shuffle', 'partially_mapped_crossover', 'linear_ranking_selection')


,Combination,ABF,STD,ABF/STD
0,"('block_rotation_mutation', 'cyclic_crossover'...",1.278492,0.041743,30.627655
1,"('block_rotation_mutation', 'cyclic_crossover'...",1.436627,0.030591,46.962975
2,"('block_rotation_mutation', 'partially_mapped_...",1.258933,0.044113,28.538627
3,"('block_rotation_mutation', 'partially_mapped_...",1.609877,0.044517,36.163397
4,"('semi_shuffle', 'cyclic_crossover', 'linear_r...",1.242853,0.043635,28.483168
5,"('semi_shuffle', 'cyclic_crossover', 'tourname...",1.427883,0.025023,57.063875
6,"('semi_shuffle', 'partially_mapped_crossover',...",1.256759,0.049787,25.242767
7,"('semi_shuffle', 'partially_mapped_crossover',...",1.563082,0.049363,31.664938
8,"('two_phase_shuffle_mutation', 'cyclic_crossov...",1.284225,0.029379,43.712327
9,"('two_phase_shuffle_mutation', 'cyclic_crossov...",1.407412,0.023752,59.255029
